# Sávdetektálás megvalósítása egyenes útvonal esetén

Az algoritmus főbb lépései:

* Bemenetként RGB színes videó megadása.
* Képkockák átalakítása szürkeárnyalatossá.
* Gauss-szűrő alkalmazásával a képzaj csökkentése.
* Canny Edge Detector alkalmazása az élek felismerésére.
* Sávvonalak várható előfordulási területének körülhatárolása.
* Vonal koordináták megtalálása.
* Hough-transzformációval egyenesek illesztése.
* Koordináták illesztése a Canny képhez.
* Sikeres detektálás.


## Szükséges kiegészítő függvénykönyvtárak importálása

In [10]:
import matplotlib.pyplot as plt #képen való grafikus ábrázolás megjelenítése
import matplotlib.image as mpimg #kép manipulációs műveletek
import numpy as np #matematikai háttér biztosítása
import cv2 #gépi látásért felelős funkciók
import math #matematikai függvények végrehajtása
import time #idővel kapcsolatos műveletek elvégzéséhez
from moviepy.editor import VideoFileClip #videóval kapcsolatos műveletek
from IPython.display import HTML #böngésző alapú notebook felület
# a sorok eredménye legyen látható futtatás során ezen a felületen
%matplotlib inline 

## A detektáló algoritmus időmérésre szolgáló kódok közé illesztve

In [11]:
start_time = time.time() #innen indul az idő mérése
" sávdetektáló algoritmus beillesztése "

#szürkeárnyalatos kép definiálása
def grayscale(img):
    #színes kép szürkeárnyalatossá konvertálása
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

#Canny (éldetektáló) definiálása küszöbértékek alkalmazásával    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

#Gauss-szűrő definiálása kernel (szűrő) méret alkalmazásával
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

#Sávvonalak várható előfordulási területének körülhatárolása
def region_of_interest(img, vertices):
    #vizsgálat céljának megfelelő terület megadása
    #2D-s üres tömb létrehozása sorok és oszlopok feltöltöséhez szürkeárnyalatos kép esetén 
    #(a távolság és szög változók tárolására (későbbi egyenes meghatározáshoz))
    #üres tömb ugyanolyan széles, magas, mint az eredeti kép
    mask = np.zeros_like(img)   
    
    #régión kívüli területek figyelmenhagyása
    if len(img.shape) > 2: #kép alakjának lekérése, sorok-oszlopok-csatornák száma (len- lista elemszámainak visszaadása)
        channel_count = img.shape[2]  #csatornaszám változóban a képadatok magasság és szélesség adatok tárolása
        ignore_mask_color = (255,) * channel_count #figyelmenkívül hagyott szín
    else:
        ignore_mask_color = 255
        
    #geometrián belül a színek meghagyása      
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #amennyiben a kép pixelei nem nullál visszaadja a képet
    masked_image = cv2.bitwise_and(img, mask) #bitenkénti operátorok alkalmazása a kép vizsgálaton kívüli részének elhagyásához
    #visszatérési érték egy tömb, mely az eredeti kép egyesítéséből adódik
    return masked_image

#Vonalak rajzolásának definiálása, szín és vastagság megadásával 
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

#Lejtésirány meghatározása 
def slope_lines(image,lines):
    
    img = image.copy()
    poly_vertices = []
    order = [0,1,3,2]

    left_lines = [] # baloldali vonalak /
    right_lines = [] # jobboldali vonalak \
    for line in lines:
        for x1,y1,x2,y2 in line:

            if x1 == x2:
                pass #Függőleges vonal illesztése
            else:
                m = (y2 - y1) / (x2 - x1) #egyenes meredeksége
                c = y1 - m * x1 #egyenes egyenlete a tengelymetszetet kifejezve

                if m < 0:
                    left_lines.append((m,c)) #ha a meredség kisebb nullánál, baloldali vonalhoz történő besorolás
                elif m >= 0:
                    right_lines.append((m,c)) #ha a meredség nagyobb vagy egyenlő, mint nulla, jobboldali vonalhoz való besorolás

    left_line = np.mean(left_lines, axis=0)
    right_line = np.mean(right_lines, axis=0)

    #print(left_line, right_line)
    
#Lejtésirány meghatározához for ciklus  
    for slope, intercept in [left_line, right_line]:

        #kép teljes magasságának megadása
        rows, cols = image.shape[:2]
        y1= int(rows) #image.shape[0]

        #azon magasság megadása ameddig a felismert sáv kitöltésre kerül(magasságmérés bal felsősaroktól lefele történik)
        #ezen érték meghatározása videónként eltérő
        y2= int(rows*0.65) #int(0.6*y1)

        #egyenes egyenletéből: y=mx +c az x kifjeezése: x=(y-c)/m
        x1=int((y1-intercept)/slope)
        x2=int((y2-intercept)/slope)
        poly_vertices.append((x1, y1))
        poly_vertices.append((x2, y2))
        draw_lines(img, np.array([[[x1,y1,x2,y2]]]))
    
    #poligon csúcsainak megadása
    poly_vertices = [poly_vertices[i] for i in order]
    cv2.fillPoly(img, pts = np.array([poly_vertices],'int32'), color = (0,255,0))
    return cv2.addWeighted(image,0.7,img,0.4,0.)

    cv2.polylines(img,np.array([poly_vertices],'int32'), True, (0,0,255), 10)
    print(poly_vertices)
    cv2.polylines(img,np.array([poly_vertices],'int32'), True, (0,0,255), 10)
    print(poly_vertices)


#Hough-transzformáció paramétereinek megadása    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    Canny transzformáció eredménye a bemeneti (img) kép
        
    Hough-vonalakkal ellátott eredmény képet ad kimentként.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)
    line_img = slope_lines(line_img,lines) #sáv kitöltése
    return line_img

#súlyozott kép definiálása
def weighted_img(img, initial_img, α=0.1, β=1., γ=0.):
    """
    Hough-transzformáció eredményeként kapott kép megadása bementként (img)
    
    `initial_img` = feldolgozás előtti eredeti bemeneti kép
    
    Eredmény kép számítása:
    
    initial_img * α + img * β + γ
    Fontos, hogy a két kép azonos méretű legyen!
    """
    lines_edges = cv2.addWeighted(initial_img, α, img, β, γ)
    #lines_edges = cv2.polylines(lines_edges,get_vertices(img), True, (0,0,255), 10) #trapéz láthatóvá tétele
    return lines_edges

#Trapéz alakú geometria csúcspontjainak paraméterezése (egyes videók esetén elétrő paraméterek szükségesek)
def get_vertices(image):
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.15, rows]
    top_left     = [cols*0.45, rows*0.65]
    bottom_right = [cols*0.95, rows]
    top_right    = [cols*0.55, rows*0.65] 
    
    ver = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return ver

#Sávkereső algoritmus definiálása
def lane_finding_pipeline(image):
    
    
    gray_img = grayscale(image)
    
    smoothed_img = gaussian_blur(img = gray_img, kernel_size = 5)
   
    canny_img = canny(img = smoothed_img, low_threshold = 180, high_threshold = 240)
   
    masked_img = region_of_interest(img = canny_img, vertices = get_vertices(image))
    
    houghed_lines = hough_lines(img = masked_img, rho = 1, theta = np.pi/180, threshold = 20, min_line_len = 20, max_line_gap = 180)
    
    output = weighted_img(img = houghed_lines, initial_img = image, α=0.8, β=1., γ=0.)
    
    return output

#Videó fájl eredményének és bemenetének elérési útjainak megadása
white_output = r'C:\Users\Eszti\Desktop\Python projects//Lane_eredmeny/L1_1203_t2.mp4'
clip1 = VideoFileClip(r'C:\Users\Eszti\Desktop\Python projects/test_videos/L1.mp4')
white_clip = clip1.fl_image(lane_finding_pipeline)
%time white_clip.write_videofile(white_output, audio=False)


#Itt ér véget az algoritmus lefutási sebességének mérése
end_time = time.time()
print("Vegrehajtasi ido: ", end_time - start_time,"s")

Moviepy - Building video C:\Users\Eszti\Desktop\Python projects//Lane_eredmeny/L1_1203_t2.mp4.
Moviepy - Writing video C:\Users\Eszti\Desktop\Python projects//Lane_eredmeny/L1_1203_t2.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\Eszti\Desktop\Python projects//Lane_eredmeny/L1_1203_t2.mp4
Wall time: 3.05 s
Vegrehajtasi ido:  3.4751596450805664 s
